This Notebook is a kaggle tutorial for Japanese kaggle beginners writen in Japanese.

# 8. Titanicの先へ行く①！ 複数テーブルを結合してみよう

この[Notebook](https://www.kaggle.com/sishihara/upura-kaggle-tutorial-08-merge-tables)では、複数のファイルを結合して機械学習アルゴリズムに入力するためのデータを用意する方法を扱います。

題材にするのは「[Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk)」[^65]コンペです。このコンペでは、融資の申込に対して、それぞれが債務不履行になるか否かを予測します。application_{train/test}.csvに含まれる申込1件に対して、過去の申込履歴などの関連情報を含むcsvファイルが複数存在するデータ構造で、どのように有益な情報を取り出すかが問われるコンペとなりました。

ここでは、公開Notebook「[Introduction to Manual Feature Engineering](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering)」の冒頭の例を引用します。

まずは主ファイルであるapplication_train.csvを読み込みます。各行が1つの融資の申込を表し、TAEGETが予測の対象です。

In [ ]:
import pandas as pd


application_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
application_train.head()

次いで、サブファイルに当たるbureau.csvを読み込みます。bureau.csvには、他の金融機関から提供された過去の申込履歴が記録されています。

図に示された通り、SK_ID_CURRで紐づく関係性です。過去の履歴なので、application_trainの1行に対して、bureauの複数行が対応する可能性があります。

In [ ]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau.head()

仮に1:1の関係ならば、純粋にデータを結合してしまえば良いです。ここでは1:Nの関係なので、N側のデータセットを何かしらの方法で集約して情報量を落とす必要があります。

「何かしらの方法で集約」の部分は、「3. ここで差がつく！ 仮説に基づいて新しい特徴量を作ってみよう」で説明したような特徴量エンジニアリングの力が試されます。思考停止でさまざまな集約パターンを試す場合もありますが、今回のデータセットのようにある程度カラム数も多い場合には、意味のある仮説を立てるのが望ましいと思います。

ここでは「過去の申込回数」が有効な特徴量になるという仮説を得たとします。この特徴量は、次のようにbureauから作成可能です。SK_ID_CURRごとに、回数が集計されています。

In [ ]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns={'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

あとは、application_trainにSK_ID_CURRをキーにして結合するだけです。

この際`how='left'`という引数が与えられている点に注目です。SQLを使ったことのある方はイメージしやすいかと思いますが、引数として与えられているデータセットのうち、左側のファイルを軸にデータセットを結合していくという意味合いです。右側のデータセットに該当する値が含まれていない場合は、欠損値になります。

仮にこの引数を指定しないと、両者に含まれるSK_ID_CURRのデータセットのみが返ります。ここでprevious_loan_countsには申込が0回のSK_ID_CURRは含まれていないので、データセットの欠落が発生する可能性があります。

application_trainの行数が減るのは学習用データセットの量を保つ観点から望ましくありません。application_test.csvを扱う場合は、予測すべきデータセットが欠落する事態を招いてしまいます。

この欠損値は、意味合いを考えると0で補完するのが適切です。


In [ ]:
application_train = pd.merge(application_train, previous_loan_counts, on='SK_ID_CURR', how='left')

# 欠損値の処理
application_train['previous_loan_counts'].fillna(0, inplace=True)
application_train.head()